In [1]:
%load_ext autoreload
%autoreload 2
    
import numpy as np
import sys
#sys.path.append('/home/benedikt/Dokumente/parabolische_inverse_probleme')
sys.path.append('/run/media/benedikt/38E84473425D444F/Dokumente/Studium/Mathematik/PhD/parabolische_inverse_probleme')
from pymor.core.logger import set_log_levels
    
from pymor.basic import *
from problems import whole_problem
from discretizer import discretize_instationary_IP
from pymor.parameters.base import ParameterSpace

from model import InstationaryModelIP


In [2]:
set_log_levels({
    'pymor' : 'WARN'
})

In [10]:
N = 100                                                                         # FE Dofs = (N+1)^2                                                
noise_level = 1e-5        
nt = 100
fine_N = 2 * N

dims = {
    'N': N,
    'nt': nt,
    'fine_N': fine_N,
    'state_dim': (N+1)**2,
    'fine_state_dim': (fine_N+1)**2,
    'diameter': np.sqrt(2)/N,
    'fine_diameter': np.sqrt(2)/fine_N,
    'par_dim': (N+1)**2,
    'output_dim': 1,                                                                                                                                                                         # options to preassemble affine components or not
}

bounds = [0.001*np.ones((dims['par_dim'],)), 10e2*np.ones((dims['par_dim'],))]

model_parameter = {
    'T_initial' : 0,
    'T_final' : 1,
    'noise_percentage' : None,
    'noise_level' : 0.00,
    'q_circ' : 3*np.ones((nt, dims['par_dim'])), 
    'q_exact' : None,
    'bounds' : bounds,
    #'parameter_space' : ParameterSpace(analytical_problem.parameters, bounds) 
    'parameters' : None
}

optimizer_parameter = {
    
}

In [11]:
print('Construct problem..')                                                     
analytical_problem, q_exact, N, problem_type, exact_analytical_problem, energy_problem = whole_problem(
                                                        N = N,
                                                        parameter_location = 'reaction',
                                                        boundary_conditions = 'dirichlet',
                                                        exact_parameter = 'Kirchner',
                                                       )


Construct problem..


In [12]:
model_parameter['q_exact'] = q_exact
model_parameter['parameters'] = analytical_problem.parameters


In [ ]:
print('Discretizing problem...')                                                
# discretize analytical problem to obtain inverse problem fom
building_blocks = discretize_instationary_IP(analytical_problem,
                            model_parameter,
                            dims, 
                            problem_type
                        ) 

model = InstationaryModelIP(
    *building_blocks,
    dims = dims,
    model_parameter = model_parameter
)


Discretizing problem...


In [ ]:
#q_0 = []
q_exact = []
for idx in range(dims['nt']):
    #q_0.append(model_parameter['q_exact'] + 10 * idx)
    q_exact.append(model_parameter['q_exact'])
q_exact = np.array(q_exact)
q_0 = q_exact.copy() 
# q_0[:,0] += 1


In [ ]:
from gradient_descent import gradient_descent
q = q_0
d = q.copy()
#d[:,:] = 0
gradient_descent(model, q, d, max_iter=10000, alpha=1e-2, tol=1e-5, inital_step_size=1e4)

In [15]:
# def compute(q):
#     u = model.solve_state(q)
#     p = model.solve_adjoint(q, u)
#     J = model.objective(u)
#     nabla_J = model.gradient(u,p)
#     return J, nabla_J

# J, nabla_J = compute(q_0)
# print(J)
# print(np.linalg.norm(nabla_J.to_numpy()))

# q = q_0
# for i in range(10000):
#     J, nabla_J = compute(q)
#     d -= nabla_J.to_numpy()
#     print(J)

In [18]:
q = q_0
d = q.copy()
#d[:,:] = 0

def compute(q, d, alpha):
    u = model.solve_state(q)
    p = model.solve_adjoint(q, u)
    
    lin_u = model.solve_linearized_state(q, d, u)
    lin_p = model.solve_linearized_adjoint(q, u, lin_u)
    
    lin_J = model.linearized_objective(q, d, u, lin_u, alpha)
    lin_nabla_J = model.linearized_gradient(q, d, u, lin_p, alpha)
    return lin_J, lin_nabla_J

lin_J, lin_nabla_J = compute(q, d, 0)
print(lin_J)
print(np.linalg.norm(lin_nabla_J.to_numpy()))
d -= 1e4 * lin_nabla_J.to_numpy()
lin_J, lin_nabla_J = compute(q, d, 0)
print(lin_J)
print(np.linalg.norm(lin_nabla_J.to_numpy()))

2.3867048368890484e-05
3.209909337166235e-06
2.3856712126759203e-05
3.209167408108818e-06


In [8]:
def compute(q, d, alpha):
    u = model.solve_state(q)
    p = model.solve_adjoint(q, u)
    
    lin_u = model.solve_linearized_state(q, d, u)
    lin_p = model.solve_linearized_adjoint(q, u, lin_u)
    
    lin_J = model.linearized_objective(q, d, u, lin_u, alpha)
    lin_nabla_J = model.linearized_gradient(q, d, u, lin_p, alpha)
    return lin_J, lin_nabla_J

q = q_0
d = q.copy()
for i in range(10000):
    lin_J, lin_nabla_J = compute(q, d, 1)
    d -= lin_nabla_J.to_numpy()
    print(lin_J)


39.72254200968959
39.44931607466045
39.177093943916105
38.9058756174908
38.63566109541878
38.3664503777343
38.0982434644716
37.83104035566493
37.56484105134853
37.299645551556644
37.03545385632353
36.772265965683424
36.51008187967057
36.24890159831923
35.988725121663634
35.729552449738044
35.471383582576685
35.21421852021381
34.95805726268369
34.70289981002054
34.448746162258615
34.195596319432155
33.94345028157543
33.69230804872266
33.44216962090811
33.19303499816601
32.94490418053062
32.69777716803618
32.45165396071693
32.20653455860713
31.962418961741022
31.719307170152845
31.477199183876845
31.236095002947277
30.99599462739839
30.756898057264415
30.518805292579607
30.281716333378213
30.04563117969447
29.810549831562646
29.576472289016966
29.343398552091685
29.111328620821048
28.880262495239293
28.65020017538068
28.42114166127945
28.19308695296985
27.966036050486128
27.73998895386252
27.51494566313328
27.290906178332662
27.067870499494894
26.845838626654242
26.624810559844935
26.404


KeyboardInterrupt



In [19]:
J, nabla_J = compute(q_0)
print(J)
print(nabla_J)

0.0012500000000000011
[[1.06904994e-09 7.52967833e-09 1.43982664e-08 ... 1.46831147e-08
  7.21949784e-09 1.03216528e-09]
 [1.19224861e-09 8.61697352e-09 1.68575706e-08 ... 1.70338053e-08
  8.18605960e-09 1.14028573e-09]
 [1.17578103e-09 8.64405623e-09 1.70945864e-08 ... 1.72289045e-08
  8.18691109e-09 1.12010707e-09]
 ...
 [8.01172210e-10 6.70063988e-09 1.39910581e-08 ... 1.42122430e-08
  6.36392196e-09 7.57160227e-10]
 [6.88481214e-10 6.03854113e-09 1.27737958e-08 ... 1.29738738e-08
  5.72016666e-09 6.46565190e-10]
 [5.25417269e-10 4.86748405e-09 1.02865415e-08 ... 1.03275491e-08
  4.52685347e-09 4.82370768e-10]]


In [11]:
np.max(nabla_J.to_numpy())

1.4728911112852965e-07

In [28]:
# q = q_0
# for i in range(10000):
#     J, nabla_J = compute(q)
#     q -= 1e5 * nabla_J.to_numpy()
#     print(J)



In [36]:
J, nabla_J = compute(q_0 - 1e1 * nabla_J.to_numpy())
print(J)
#print(nabla_J)

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

0.00016074369508906668


In [59]:
J, nabla_J = compute(q_exact)
print(J)
#print(nabla_J)

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

-8.131516293641283e-20
